In [1]:
data_file = "Data/CNM_004.csv"
data_file_header_lines = 2
data_file_fields_separator = ","
# Argument: Block number starting from 0 (number)
classes_file_template = "Data/Day2_%d.tsv"
classes_files_num = 5
classes_file_header_lines = 1
classes_file_fields_separator = "\t"
classes_file_timestamp_shift = 5.05
classes_output_file = "Data/Classes.csv"

In [2]:
def read_file(file, header_lines, separator):
    headers = []
    data = {}
    with open(file, "r") as f_file:
        line_num = 0
        for line in f_file:
            line_num += 1
            header = False
            if line_num <= header_lines:
                header = True
            line = line.replace(separator + " ", separator).strip(" \n\r")
            if len(line) > 0:
                row = line.split(separator)
                values = []
                values_good = True
                if header:
                     headers.append(row)
                else:
                    timestamp_str = row[0]
                    timestamp = -1
                    try:
                        timestamp = float(timestamp_str)
                    except:
                        print("Error timestamp value '%s' in line #%d" % (timestamp_str, line_num))
                    if timestamp >= 0:
                        payload = row[1:]
                        data[timestamp] = payload
    return {"headers": headers, "data": data}

In [3]:
__data = read_file(data_file, data_file_header_lines, data_file_fields_separator)
data = __data["data"]
headers = __data["headers"][0][1:]
print("Data points: %d" % (len(data)))
print("Features: %d" % (len(headers)))
data_block_offsets = []
prev_timestamp = None
cur_idx = 0
for timestamp in data:
    if prev_timestamp == None or timestamp - prev_timestamp > 1:
        data_block_offsets.append(cur_idx)
    cur_idx += 1
    prev_timestamp = timestamp
print("Data blocks: %d" % (len(data_block_offsets)))

Data points: 30025
Features: 92
Data blocks: 5


In [4]:
with open(classes_output_file, "w") as f_file:
    f_file.write("class_id, block_#, instance_#, " + (", ".join(headers)) + "\n")
    for classes_file_idx in range(classes_files_num):
        classes_file = classes_file_template % (classes_file_idx)
        __classes = read_file(classes_file, classes_file_header_lines, classes_file_fields_separator)
        classes = __classes["data"]
        class_ids = __classes["headers"][0][1:]
        block = classes_file_idx
        print("Block #%d has %d annotations with %d classes: %s" % (block, len(classes), len(class_ids), str(class_ids)))
        classes_running = [False for col in range(len(class_ids))]
        classes_instance = [-1 for col in range(len(class_ids))]
        data_offset = data_block_offsets[block]
        data_size = (data_block_offsets[block + 1] if block < len(data_block_offsets) - 1 else len(data)) - data_block_offsets[block]
        classes_timestamps = list(classes.keys())
        cur_classes_idx = 0
        data_block_timestamps = list(data.keys())[data_offset:data_offset+data_size]
        data_block_timestamp_shift = data_block_timestamps[0]
        #print("%f, %f" % (data_block_timestamp_shift, classes_file_timestamp_shift))
        data_idx = 0
        for data_timestamp in data_block_timestamps:
            while cur_classes_idx < len(classes_timestamps) and (data_timestamp - data_block_timestamp_shift > classes_timestamps[cur_classes_idx] - classes_file_timestamp_shift or classes_timestamps[cur_classes_idx] - classes_file_timestamp_shift < 0):
                cur_classes_idx += 1
            if cur_classes_idx >= len(classes_timestamps):
                break
            #print("%f => %f [%d]" % (data_timestamp - data_block_timestamp_shift, classes_timestamps[cur_classes_idx] - classes_file_timestamp_shift, cur_classes_idx))
            for class_idx in range(len(class_ids)):
                if classes[classes_timestamps[cur_classes_idx]][class_idx] != "0":
                    if not classes_running[class_idx]:
                        classes_instance[class_idx] += 1
                        classes_running[class_idx] = True
                    f_file.write(("%s, %d, %d, " % (class_ids[class_idx], block, classes_instance[class_idx])) + (", ".join(data[data_timestamp])) + "\n")
                else:
                    if classes_running[class_idx]:
                        classes_running[class_idx] = False

Block #0 has 6100 annotations with 2 classes: ['1', '2']
Block #1 has 6101 annotations with 4 classes: ['3', '4', '5', '6']
Block #2 has 6100 annotations with 4 classes: ['3', '4', '5', '6']
Block #3 has 6100 annotations with 4 classes: ['3', '4', '5', '6']
Block #4 has 6101 annotations with 4 classes: ['3', '4', '5', '6']
